In [124]:
import pandas as pd
import numpy as np
import requests
import yaml
import time
from random import shuffle
import json
from pymongo import MongoClient
from collections import Counter
from geopy.geocoders import Nominatim
geolocator = Nominatim()
pd.set_option('display.max_columns', 200)

In [125]:
def create_pandas_df_from_json(path):
    '''
    INPUT: filepath string
    OUTPUT: pandas database
    '''
    return pd.read_json(file_path, lines=True)

def is_food(item):
    '''
    INPUT: cell from pandas dataframe
    OUTPUT: boolean
    '''
    restaurants_and_related_categories = ['Restaurants', 'Italian','Food', 'Bars','Fast Food', 'Coffee & Tea', 'Sandwiches']
    if len(set(restaurants_and_related_categories) & set(item)) >= 1:
        return True
    else:
        return False
    
def flatten_dict(row):
    out = {}
    for key, value in row.items():
        if type(value) != dict:
            out[key] = value
        else:
            sub_key = key
            for k, v in value.items():
                out[sub_key + "|" + k] = v
    return out

def make_exists_function(key):
    def get_key_if_exists(row):
        if key in row:
            return row[key]
        else:
            return "N/A"
    return get_key_if_exists

def add_restaurant_count_column(dataframe):
    restaurant_frequency = dataframe.groupby(['name']).count().sort_values('address', ascending=False)

    restaurant_frequency = pd.DataFrame(restaurant_frequency['address'])

    restaurant_frequency.columns = ['restaurant_count']

    restaurant_frequency['name'] = restaurant_frequency.index

    restaurant_frequency = restaurant_frequency[['name', 'restaurant_count']]

    return previously_open_US_restaurants.merge(restaurant_frequency, how='left', left_on='name', right_on='name')

def closed_on_google(row):
    try:
        return row[0]['permanently_closed']
    except:
        return False
    
def fix_percent(row):
    row = str(row).strip('%')
    row = float(row)
    return row/100

In [ ]:
file_path = 'https://s3-us-west-2.amazonaws.com/businesspredictiondata/business.json'
yelp_business_data = create_pandas_df_from_json(file_path)

In [128]:
#filters businesses that were open when this dataset was published Jan. 2018
open_businesses = yelp_business_data[yelp_business_data['is_open'] == 1]

#creates column that says if business is restaurant and creates df of just open restaurants
open_businesses['is_food'] = open_businesses['categories'].apply(is_food)
open_restaurants = open_businesses[open_businesses['is_food'] == True]

#creates column that says if business is in USA and creates df of just
#restaurants open in the US as of January 2018
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA",
      "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
      "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
      "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
      "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
open_restaurants['in_US'] = open_restaurants['state'].isin(states)
previously_open_US_restaurants = open_restaurants[open_restaurants['in_US'] == True]

#creates dummy columns for 
previously_open_US_restaurants['flat_attributes'] = previously_open_US_restaurants['attributes'].apply(flatten_dict)
all_attributes = []
for row in previously_open_US_restaurants['flat_attributes']:
    all_attributes.extend(row.keys())
unique_attributes = list(dict(Counter(all_attributes).most_common(50)).keys())

for key in unique_attributes:
    previously_open_US_restaurants['Attribute|has_'+key] = previously_open_US_restaurants['flat_attributes'].apply(lambda x: key in x)
    
    f = make_exists_function(key)
    previously_open_US_restaurants['Attribute|' +key + ' value:'] = previously_open_US_restaurants['flat_attributes'].apply(f)
    
all_categories = []
[all_categories.extend(item) for item in list(previously_open_US_restaurants['categories'])]

most_common_categories = list(dict(Counter(all_categories).most_common(50)).keys())

for key in most_common_categories:
    previously_open_US_restaurants[f"Category|{key}_true"] = previously_open_US_restaurants['categories'].apply(lambda x: key in x)

previously_open_US_restaurants = add_restaurant_count_column(previously_open_US_restaurants)
    
client = MongoClient('mongodb://localhost:27017/')
restaurants = client['restaurants']
google_places = restaurants['google_places']
start_time = time.time()

google_df = pd.DataFrame(list(google_places.find()))

google_df = google_df[['queried_name', 'yelp_business_id', 'results']]

google_df['closed_on_google'] = google_df['results'].apply(closed_on_google)

restaurants_with_google_data = previously_open_US_restaurants.merge(google_df, how='inner', left_on='business_id', right_on='yelp_business_id')

#removes rows without any matching data from Google
restaurants_with_google_data = restaurants_with_google_data[restaurants_with_google_data['results'].map(len) > 0]

zip_code_df = pd.read_csv('/Users/ElliottC/g/projects/yelp/predicting_restaurant_closure/data/zip_code_data.csv')

zip_code_df['Zip Code'] = zip_code_df['Zip Code'].apply(str)

restaurants_with_economic_data = restaurants_with_google_data.merge(zip_code_df, how='left', left_on='postal_code', right_on='Zip Code')

restaurants_with_economic_data.iloc[:,-19:] = restaurants_with_economic_data.iloc[:,-19:].fillna(0).copy()

percent_columns = ['Educational Attainment: Percent high school graduate or higher', 'Individuals below poverty level']
for col in percent_columns:
    restaurants_with_economic_data[col] = restaurants_with_economic_data[col].apply(fix_percent)

num_columns = ['2016 ACS 5-Year Population Estimate',
 'American Indian and Alaska Native alone',
 'Asian alone',
 'Black or African American alone',
 'Census 2010 Total Population',
 'Foreign Born Population',
 'Hispanic or Latino (of any race)',
 'Median Age',
 'Median Household Income',
 'Native Hawaiian and Other Pacific Islander alone',
 'Some Other Race alone',
 'Total housing units',
 'Two or More Races',
 'Veterans',
 'White alone',
 'White alone, Not Hispanic or Latino']
    
for col in num_columns:
    restaurants_with_economic_data[col] = restaurants_with_economic_data[col].apply(int)

restaurants_with_economic_data.to_csv('/Users/ElliottC/g/projects/yelp/predicting_restaurant_closure/data/featurized_dataframe.csv')

/Users/ElliottC/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/ElliottC/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/ElliottC/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

In [ ]:
cols = [
 'restaurant_count','review_count','stars',
    'Attribute|has_BusinessAcceptsCreditCards', 'Attribute|BusinessAcceptsCreditCards value:', 'Attribute|has_RestaurantsPriceRange2', 'Attribute|RestaurantsPriceRange2 value:', 'Attribute|has_RestaurantsTakeOut', 'Attribute|RestaurantsTakeOut value:', 'Attribute|has_BusinessParking|garage', 'Attribute|BusinessParking|garage value:', 'Attribute|has_BusinessParking|street', 'Attribute|BusinessParking|street value:', 'Attribute|has_BusinessParking|lot', 'Attribute|BusinessParking|lot value:', 'Attribute|has_BusinessParking|valet', 'Attribute|BusinessParking|valet value:', 'Attribute|has_BusinessParking|validated', 'Attribute|BusinessParking|validated value:', 'Attribute|has_BikeParking', 'Attribute|BikeParking value:', 'Attribute|has_OutdoorSeating', 'Attribute|OutdoorSeating value:', 'Attribute|has_RestaurantsGoodForGroups', 'Attribute|RestaurantsGoodForGroups value:', 'Attribute|has_RestaurantsDelivery', 'Attribute|RestaurantsDelivery value:', 'Attribute|has_RestaurantsReservations', 'Attribute|RestaurantsReservations value:', 'Attribute|has_GoodForKids', 'Attribute|GoodForKids value:', 'Attribute|has_HasTV', 'Attribute|HasTV value:', 'Attribute|has_WiFi', 'Attribute|has_Ambience|romantic', 'Attribute|Ambience|romantic value:', 'Attribute|has_Ambience|intimate', 'Attribute|Ambience|intimate value:', 'Attribute|has_Ambience|classy', 'Attribute|Ambience|classy value:', 'Attribute|has_Ambience|hipster', 'Attribute|Ambience|hipster value:', 'Attribute|has_Ambience|touristy', 'Attribute|Ambience|touristy value:', 'Attribute|has_Ambience|trendy', 'Attribute|Ambience|trendy value:', 'Attribute|has_Ambience|upscale', 'Attribute|Ambience|upscale value:', 'Attribute|has_Ambience|casual', 'Attribute|Ambience|casual value:', 'Attribute|has_Ambience|divey', 'Attribute|Ambience|divey value:', 'Attribute|has_Alcohol', 'Attribute|has_RestaurantsAttire', 'Attribute|has_GoodForMeal|dessert', 'Attribute|GoodForMeal|dessert value:', 'Attribute|has_GoodForMeal|latenight', 'Attribute|GoodForMeal|latenight value:', 'Attribute|has_GoodForMeal|lunch', 'Attribute|GoodForMeal|lunch value:', 'Attribute|has_GoodForMeal|dinner', 'Attribute|GoodForMeal|dinner value:', 'Attribute|has_GoodForMeal|breakfast', 'Attribute|GoodForMeal|breakfast value:', 'Attribute|has_GoodForMeal|brunch', 'Attribute|GoodForMeal|brunch value:', 'Attribute|has_Caters', 'Attribute|Caters value:', 'Attribute|has_NoiseLevel', 'Attribute|has_RestaurantsTableService', 'Attribute|RestaurantsTableService value:', 'Attribute|has_WheelchairAccessible', 'Attribute|WheelchairAccessible value:', 'Attribute|has_HappyHour', 'Attribute|HappyHour value:', 'Attribute|has_GoodForDancing', 'Attribute|GoodForDancing value:', 'Attribute|has_DriveThru', 'Attribute|DriveThru value:', 'Attribute|has_Music|dj', 'Attribute|Music|dj value:', 'Attribute|has_Music|background_music', 'Attribute|Music|background_music value:', 'Attribute|has_Music|no_music', 'Attribute|Music|no_music value:', 'Attribute|has_Music|karaoke', 'Attribute|Music|karaoke value:', 'Attribute|has_Music|live', 'Attribute|Music|live value:', 'Attribute|has_Music|video', 'Attribute|Music|video value:', 'Attribute|has_Music|jukebox', 'Attribute|Music|jukebox value:', 'Attribute|has_CoatCheck', 'Attribute|CoatCheck value:', 'Attribute|has_DogsAllowed', 'Attribute|DogsAllowed value:', 'Attribute|has_Smoking', 'Category|Restaurants_true', 'Category|Food_true', 'Category|Nightlife_true', 'Category|Bars_true', 'Category|Fast Food_true', 'Category|American (Traditional)_true', 'Category|Sandwiches_true', 'Category|Pizza_true', 'Category|Mexican_true', 'Category|Burgers_true', 'Category|American (New)_true', 'Category|Breakfast & Brunch_true', 'Category|Coffee & Tea_true', 'Category|Grocery_true', 'Category|Italian_true', 'Category|Specialty Food_true', 'Category|Shopping_true', 'Category|Chinese_true', 'Category|Event Planning & Services_true', 'Category|Chicken Wings_true', 'Category|Salad_true', 'Category|Bakeries_true', 'Category|Desserts_true', 'Category|Convenience Stores_true', 'Category|Ice Cream & Frozen Yogurt_true', 'Category|Sports Bars_true', 'Category|Seafood_true', 'Category|Beer_true', 'Category|Wine & Spirits_true', 'Category|Caterers_true', 'Category|Delis_true', 'Category|Cafes_true', 'Category|Drugstores_true', 'Category|Japanese_true', 'Category|Arts & Entertainment_true', 'Category|Juice Bars & Smoothies_true', 'Category|Pubs_true', 'Category|Steakhouses_true', 'Category|Sushi Bars_true', 'Category|Asian Fusion_true', 'Category|Barbeque_true', 'Category|Diners_true', 'Category|Lounges_true', 'Category|Automotive_true', 'Category|Gas Stations_true', 'Category|Cocktail Bars_true', 'Category|Mediterranean_true', 'Category|Wine Bars_true', 'Category|Food Trucks_true', 'Category|Tex-Mex_true']

In [ ]:
col_vals = {}
for col in cols:
    col_vals[col] = restaurants_with_google_data[col].value_counts()

non_binary = {}
binary = {}
binary_with_na = {}
for key, value in col_vals.items():
    if (list(set(col_vals[key].index))[0] != False) and (list(set(col_vals[key].index))[0] != True):
        non_binary[key] = list(col_vals[key].index)
    elif len(list(set(col_vals[key].index))) > 2:
        binary[key] = list(col_vals[key].index)
    else:
        binary_with_na[key] = list(col_vals[key].index)

In [109]:
percent_columns = ['Educational Attainment: Percent high school graduate or higher', 'Individuals below poverty level']
for col in percent_columns:
    restaurants_with_economic_data[col] = restaurants_with_economic_data[col].apply(fix_percent)

In [110]:
def fix_percent(row):
    row = str(row).strip('%')
    row = float(row)
    return row/100

In [111]:
restaurants_with_economic_data.iloc[:,-19:] = restaurants_with_economic_data.iloc[:,-19:].fillna(0).copy()

In [99]:
num_columns = ['2016 ACS 5-Year Population Estimate',
 'American Indian and Alaska Native alone',
 'Asian alone',
 'Black or African American alone',
 'Census 2010 Total Population',
 'Foreign Born Population',
 'Hispanic or Latino (of any race)',
 'Median Age',
 'Median Household Income',
 'Native Hawaiian and Other Pacific Islander alone',
 'Some Other Race alone',
 'Total housing units',
 'Two or More Races',
 'Veterans',
 'White alone',
 'White alone, Not Hispanic or Latino']

In [120]:
for col in num_columns:
    restaurants_with_economic_data[col] = restaurants_with_economic_data[col].apply(int)

In [121]:
restaurants_with_economic_data.to_csv('/Users/ElliottC/g/projects/yelp/predicting_restaurant_closure/data/featurized_dataframe.csv')